In [31]:
import pandas as pd
import numpy as np
from openai import OpenAI
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import re

In [32]:
openai_api_key = ''

In [33]:
client = OpenAI(api_key=openai_api_key)
articles_df = pd.DataFrame(pd.read_csv('../data/19.csv'))
articles_df

,id,title,category,tags
0,1,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),INVESTMENT,"돈불리기,etf,비법,꿀팁,주식,재테크,연금저축,투자,미국주식"
1,7,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,INVESTMENT,"부동산,재테크,투자"
2,20,애들건강 음식\nㅕ,RELATIONSHIP,"반나나,채소,삶은고기,애호박,요플레"
3,22,운동\n,HOBBY,테그
4,51,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,SIDE_JOB,"전망,웹소설,성공비결,공동창작,콘텐츠,성공공식,집단창작"
5,52,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,SIDE_JOB,"세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품"
6,53,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림",SIDE_JOB,"웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품"
7,54,웹소설을 같이 쓴다는 것,SIDE_JOB,"1등,웹소설,카카오페이지"
8,55,카카오페이지 웹소설 1위 찍다!,SIDE_JOB,"1등,웹소설,카카오페이지,인기작품"
9,64,발로란트,GAME,"발로란트,세이지,게임,로터스"


In [34]:
articles_df["processed_text"] = articles_df.apply(lambda x: re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "",
                                                                   f"{x['title']} {' '.join(x['tags'].split(','))}"),
                                                  axis=1)


In [35]:
stop_words = pd.read_csv('../data/stop_words_ko.csv')
stop_words


,stop_words
0,아
1,휴
2,아이구
3,아이쿠
4,아이고
...,...
770,원
771,잘
772,통하
773,소리


In [36]:
from nltk.tokenize import word_tokenize

articles_df["processed_text"] = (
    articles_df.apply(lambda x: ' '.join(
        [word for word in word_tokenize(x['processed_text']) if word not in stop_words['stop_words'].tolist()]),
                      axis=1))

articles_df

,id,title,category,tags,processed_text
0,1,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),INVESTMENT,"돈불리기,etf,비법,꿀팁,주식,재테크,연금저축,투자,미국주식",한국에서 돈을 가장 빨리 불리는 비법 feat13월의 월급 두둑해짐 돈불리기 etf...
1,7,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,INVESTMENT,"부동산,재테크,투자",대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기 부동산 재테크 투자
2,20,애들건강 음식\nㅕ,RELATIONSHIP,"반나나,채소,삶은고기,애호박,요플레",애들건강 음식 반나나 채소 삶은고기 애호박 요플레
3,22,운동\n,HOBBY,테그,운동 테그
4,51,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,SIDE_JOB,"전망,웹소설,성공비결,공동창작,콘텐츠,성공공식,집단창작",웹소설 전망 앞으로 웹소설을 써서 성공하기는 힘들 겁니다 전망 웹소설 성공비결 공동...
5,52,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,SIDE_JOB,"세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품",웹소설로 돈 벌려면 3가지를 익히셔야 합니다 세컨드잡 웹소설 성공비결 N잡러 N잡 ...
6,53,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림",SIDE_JOB,"웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품",웹소설 쓰면 나아지는지 보여드림 웹소설 부업 성공비결 N잡 N잡러 성공공식 인기작품
7,54,웹소설을 같이 쓴다는 것,SIDE_JOB,"1등,웹소설,카카오페이지",웹소설을 쓴다는 1등 웹소설 카카오페이지
8,55,카카오페이지 웹소설 1위 찍다!,SIDE_JOB,"1등,웹소설,카카오페이지,인기작품",카카오페이지 웹소설 1위 찍다 1등 웹소설 카카오페이지 인기작품
9,64,발로란트,GAME,"발로란트,세이지,게임,로터스",발로란트 발로란트 세이지 게임 로터스


In [37]:
def get_embedding(text: str, model="text-embedding-3-large"):
    text = text.replace('\n', ' ')
    return client.embeddings.create(input=[text], model=model).data[0].embedding


articles_df['embedding'] = articles_df.apply(lambda x: get_embedding(x['processed_text']), axis=1)


# articles_df['embedding'] = articles_df.apply(lambda x: get_embedding(f"{x['title']} - {' '.join(x['tags'].split(','))}" ), axis=1)


# with open( "embedded-article-with-tag-content", "rb" ) as file:
#     articles_df = pickle.load(file)


In [38]:
from sklearn.cluster import KMeans

clusters = KMeans(random_state=0).fit_predict(articles_df['embedding'].tolist())
articles_df['cluster'] = clusters


In [39]:

with open("embedded-article-with-tag-category-large", "wb") as file:
    pickle.dump(articles_df, file)

articles_df

,id,title,category,tags,processed_text,embedding,cluster
0,1,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),INVESTMENT,"돈불리기,etf,비법,꿀팁,주식,재테크,연금저축,투자,미국주식",한국에서 돈을 가장 빨리 불리는 비법 feat13월의 월급 두둑해짐 돈불리기 etf...,"[0.012530412524938583, -0.040022511035203934, ...",2
1,7,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,INVESTMENT,"부동산,재테크,투자",대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기 부동산 재테크 투자,"[-0.004191316198557615, -0.01968361809849739, ...",2
2,20,애들건강 음식\nㅕ,RELATIONSHIP,"반나나,채소,삶은고기,애호박,요플레",애들건강 음식 반나나 채소 삶은고기 애호박 요플레,"[-0.02343979850411415, -0.01796853356063366, -...",5
3,22,운동\n,HOBBY,테그,운동 테그,"[-0.0049162860959768295, -0.020620351657271385...",7
4,51,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,SIDE_JOB,"전망,웹소설,성공비결,공동창작,콘텐츠,성공공식,집단창작",웹소설 전망 앞으로 웹소설을 써서 성공하기는 힘들 겁니다 전망 웹소설 성공비결 공동...,"[-0.02818647027015686, -0.041322916746139526, ...",6
5,52,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,SIDE_JOB,"세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품",웹소설로 돈 벌려면 3가지를 익히셔야 합니다 세컨드잡 웹소설 성공비결 N잡러 N잡 ...,"[0.024900013580918312, -0.05570428445935249, -...",1
6,53,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림",SIDE_JOB,"웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품",웹소설 쓰면 나아지는지 보여드림 웹소설 부업 성공비결 N잡 N잡러 성공공식 인기작품,"[0.001184988534078002, -0.04099058732390404, -...",1
7,54,웹소설을 같이 쓴다는 것,SIDE_JOB,"1등,웹소설,카카오페이지",웹소설을 쓴다는 1등 웹소설 카카오페이지,"[0.004231337457895279, -0.03767669200897217, -...",1
8,55,카카오페이지 웹소설 1위 찍다!,SIDE_JOB,"1등,웹소설,카카오페이지,인기작품",카카오페이지 웹소설 1위 찍다 1등 웹소설 카카오페이지 인기작품,"[0.005593918729573488, -0.04929027706384659, -...",1
9,64,발로란트,GAME,"발로란트,세이지,게임,로터스",발로란트 발로란트 세이지 게임 로터스,"[-0.02589326724410057, -0.030540088191628456, ...",7


In [52]:
from pinecone import Pinecone
from pinecone import PodSpec

pc = Pinecone(api_key=' ')

index_name = 'vectordbprac'
dimension = 3072
metric = 'euclidean'
spec = PodSpec('gcp-starter')

if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

pc.create_index(index_name, dimension=dimension, metric=metric, spec=spec)

index = pc.Index(index_name)

index_stats = index.describe_index_stats()
print(index_stats)

{'dimension': 3072,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [53]:
list_of_dict = articles_df.apply(lambda row: {
    'id': str(row['id']),
    'values': row['embedding'],
    'metadata': {
        'category': row['category'],
        'tags': row['tags'],
        'cluster' : row['cluster'],
        'title' : row['title']
    }}, axis=1).to_list()

list_of_dict

[{'id': '1',
  'values': [0.012530412524938583,
   -0.040022511035203934,
   -0.004083293955773115,
   -0.021669510751962662,
   -0.021307935938239098,
   0.024823931977152824,
   -0.027404822409152985,
   -0.0020884021650999784,
   -0.014974155463278294,
   0.028252651914954185,
   -0.04678020626306534,
   -0.011339711956679821,
   0.007636694237589836,
   -0.05615619942545891,
   0.016133686527609825,
   0.013627603650093079,
   0.03967340663075447,
   -0.007568120025098324,
   0.0023284126073122025,
   -0.01750517450273037,
   -0.0034349539782851934,
   0.004759686533361673,
   0.00833490677177906,
   -0.0022380189038813114,
   -0.011825966648757458,
   0.0035409326665103436,
   0.023128274828195572,
   0.0031466297805309296,
   -0.020285554230213165,
   -0.01346551813185215,
   0.03663119673728943,
   0.00917026773095131,
   -0.016819430515170097,
   -0.028851119801402092,
   0.02385142259299755,
   0.020111000165343285,
   -0.0221682321280241,
   0.024225464090704918,
   0.0303472

In [54]:
index.upsert(
    vectors=list_of_dict
)


{'upserted_count': 31}

In [71]:
query_test_description = word_tokenize(re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", "연애 잘 하고 싶은데"))
query_test_description = ' '.join([word for word in query_test_description if not word in stop_words])

query_embedding = get_embedding(query_test_description)


In [72]:

index.query(
    vector=query_embedding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

{'matches': [{'id': '114',
              'metadata': {'category': 'RELATIONSHIP',
                           'cluster': 5.0,
                           'tags': '응원해요,연애꿀팁,비법거래소,대처법,비법,가스라이팅,연애비법,연애,극복,연애노하우',
                           'title': '가스라이팅 해대는 애인 참교육할 수 있는 단 1가지 방법'},
              'score': 1.16317201,
              'values': []},
             {'id': '70',
              'metadata': {'category': 'CAREER',
                           'cluster': 6.0,
                           'tags': '일러스트레이터,부업,자기계발,N잡,커리어,수익화,디자이너,자기개발,프리랜서',
                           'title': '조직 밖에서 스스로 돈벌기, 절반의 성공과 절반의 실패'},
              'score': 1.53534186,
              'values': []},
             {'id': '52',
              'metadata': {'category': 'SIDE_JOB',
                           'cluster': 1.0,
                           'tags': '세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품',
                           'title': '웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다'},
              'score': 1.54223228,
              'values': []}],
 

In [60]:
articles_df

,id,title,category,tags,processed_text,embedding,cluster
0,1,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),INVESTMENT,"돈불리기,etf,비법,꿀팁,주식,재테크,연금저축,투자,미국주식",한국에서 돈을 가장 빨리 불리는 비법 feat13월의 월급 두둑해짐 돈불리기 etf...,"[0.012530412524938583, -0.040022511035203934, ...",2
1,7,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,INVESTMENT,"부동산,재테크,투자",대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기 부동산 재테크 투자,"[-0.004191316198557615, -0.01968361809849739, ...",2
2,20,애들건강 음식\nㅕ,RELATIONSHIP,"반나나,채소,삶은고기,애호박,요플레",애들건강 음식 반나나 채소 삶은고기 애호박 요플레,"[-0.02343979850411415, -0.01796853356063366, -...",5
3,22,운동\n,HOBBY,테그,운동 테그,"[-0.0049162860959768295, -0.020620351657271385...",7
4,51,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,SIDE_JOB,"전망,웹소설,성공비결,공동창작,콘텐츠,성공공식,집단창작",웹소설 전망 앞으로 웹소설을 써서 성공하기는 힘들 겁니다 전망 웹소설 성공비결 공동...,"[-0.02818647027015686, -0.041322916746139526, ...",6
5,52,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,SIDE_JOB,"세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품",웹소설로 돈 벌려면 3가지를 익히셔야 합니다 세컨드잡 웹소설 성공비결 N잡러 N잡 ...,"[0.024900013580918312, -0.05570428445935249, -...",1
6,53,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림",SIDE_JOB,"웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품",웹소설 쓰면 나아지는지 보여드림 웹소설 부업 성공비결 N잡 N잡러 성공공식 인기작품,"[0.001184988534078002, -0.04099058732390404, -...",1
7,54,웹소설을 같이 쓴다는 것,SIDE_JOB,"1등,웹소설,카카오페이지",웹소설을 쓴다는 1등 웹소설 카카오페이지,"[0.004231337457895279, -0.03767669200897217, -...",1
8,55,카카오페이지 웹소설 1위 찍다!,SIDE_JOB,"1등,웹소설,카카오페이지,인기작품",카카오페이지 웹소설 1위 찍다 1등 웹소설 카카오페이지 인기작품,"[0.005593918729573488, -0.04929027706384659, -...",1
9,64,발로란트,GAME,"발로란트,세이지,게임,로터스",발로란트 발로란트 세이지 게임 로터스,"[-0.02589326724410057, -0.030540088191628456, ...",7


In [61]:
query_test_description = word_tokenize(re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", "부자 되는 법"))
query_test_description = ' '.join([word for word in query_test_description if not word in stop_words])

query_embedding = get_embedding(query_test_description)

articles_df['similarity'] = articles_df['embedding'].apply(
    lambda x: cosine_similarity(np.array(x).reshape(1, -1), np.array(query_embedding).reshape(1, -1))[0][0])
selected_df = articles_df.sort_values(by='similarity', ascending=False)

print("검색어 :" + query_test_description)
selected_df

검색어 :부자 되는 법


,id,title,category,tags,processed_text,embedding,cluster,similarity
27,100,직장인이 부동산 부자되는 궁극의 7단계 전략,INVESTMENT,"직장인투자,부동산,부동산강의,투자관리,부자되는법,투자방법,부동산투자,부자되기",직장인이 부동산 부자되는 궁극의 7단계 전략 직장인투자 부동산 부동산강의 투자관리 ...,"[0.02064657025039196, -0.043829575181007385, -...",2,0.533959
0,1,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),INVESTMENT,"돈불리기,etf,비법,꿀팁,주식,재테크,연금저축,투자,미국주식",한국에서 돈을 가장 빨리 불리는 비법 feat13월의 월급 두둑해짐 돈불리기 etf...,"[0.012530412524938583, -0.040022511035203934, ...",2,0.474488
28,106,고객 스스로 돈을 내는 서비스를 만드는 법!,INVESTMENT,"서비스,고객,고민,유료화,재테크,상담,돈버는법",고객 스스로 돈을 내는 서비스를 만드는 법 서비스 고객 고민 유료화 재테크 상담 돈버는법,"[-0.007273979019373655, -0.04283357411623001, ...",4,0.397271
6,53,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림",SIDE_JOB,"웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품",웹소설 쓰면 나아지는지 보여드림 웹소설 부업 성공비결 N잡 N잡러 성공공식 인기작품,"[0.001184988534078002, -0.04099058732390404, -...",1,0.391024
22,93,웹소설 진단 서비스 - 카카오페이지 1등이 웹소설로 돈 버는 법 알려드림,SIDE_JOB,웹소설,웹소설 진단 서비스 카카오페이지 1등이 웹소설로 돈 버는 법 알려드림 웹소설,"[0.006804273929446936, -0.05656092241406441, -...",1,0.388197
5,52,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,SIDE_JOB,"세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품",웹소설로 돈 벌려면 3가지를 익히셔야 합니다 세컨드잡 웹소설 성공비결 N잡러 N잡 ...,"[0.024900013580918312, -0.05570428445935249, -...",1,0.374844
1,7,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,INVESTMENT,"부동산,재테크,투자",대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기 부동산 재테크 투자,"[-0.004191316198557615, -0.01968361809849739, ...",2,0.348751
14,70,"조직 밖에서 스스로 돈벌기, 절반의 성공과 절반의 실패",CAREER,"일러스트레이터,부업,자기계발,N잡,커리어,수익화,디자이너,자기개발,프리랜서",조직 밖에서 스스로 돈벌기 절반의 성공과 절반의 실패 일러스트레이터 부업 자기계발 ...,"[-0.0031810293439775705, -0.006435565650463104...",6,0.336952
23,94,직장인 부업 추천 - 블로그를 시작하세요 :),SIDE_JOB,"직장인부업,블로그,부업,네이버블로그",직장인 부업 추천 블로그를 시작하세요 직장인부업 블로그 부업 네이버블로그,"[0.02812519483268261, -0.031949013471603394, -...",2,0.321741
17,77,맥을 똑똑하게쓰기,HOBBY,"맥북,돈값해야지,맥북비싸",맥을 똑똑하게쓰기 맥북 돈값해야지 맥북비싸,"[0.022919395938515663, -0.01860092207789421, -...",0,0.300534


In [39]:
def calculate_diversity(df, column_name):
    # 각각의 임베딩끼리 모두 pairwise cosine similarity를 계산
    embeddings = np.vstack(df[column_name])
    cosine_sim = cosine_similarity(embeddings)

    # self-comparisons (diagonal elements)를 제외하고 cosine similarity 계산
    np.fill_diagonal(cosine_sim, np.nan)  # 본인과의 similarity는 제외
    avg_distance = np.nanmean(cosine_sim)

    return cosine_sim, avg_distance


diversity_score = {k: calculate_diversity(articles_df.loc[articles_df['cluster'] == k], 'embedding')[1] for k in
                   range(0, articles_df['cluster'].drop_duplicates().shape[0])}
diversity_score

/var/folders/yk/wkvyrshx24sdfg84jy4w_xnc0000gn/T/ipykernel_46783/2693057421.py:9: RuntimeWarning: Mean of empty slice
  avg_distance = np.nanmean(cosine_sim)
/var/folders/yk/wkvyrshx24sdfg84jy4w_xnc0000gn/T/ipykernel_46783/2693057421.py:9: RuntimeWarning: Mean of empty slice
  avg_distance = np.nanmean(cosine_sim)
/var/folders/yk/wkvyrshx24sdfg84jy4w_xnc0000gn/T/ipykernel_46783/2693057421.py:9: RuntimeWarning: Mean of empty slice
  avg_distance = np.nanmean(cosine_sim)


{0: 0.3309789762790128,
 1: 0.42707571943744155,
 2: 0.38965886311723325,
 3: nan,
 4: nan,
 5: 0.25924281901746665,
 6: 0.25379693613038995,
 7: nan}

In [31]:
cluster = articles_df.loc[articles_df['cluster'] == 2]
cluster

,title,tags,category,processed_text,embedding,cluster,similarity
0,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),"돈불리기,etf,비법,꿀팁,주식,재테크,연금저축,투자,미국주식",INVESTMENT,한국에서 돈을 가장 빨리 불리는 비법 feat13월의 월급 두둑해짐 돈불리기 etf...,"[0.012505175545811653, -0.04002154991030693, -...",2,0.404299
1,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,"부동산,재테크,투자",INVESTMENT,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기 부동산 재테크 투자,"[-0.003373168408870697, -0.01604032889008522, ...",2,0.308757
23,직장인 부업 추천 - 블로그를 시작하세요 :),"직장인부업,블로그,부업,네이버블로그",SIDE_JOB,직장인 부업 추천 블로그를 시작하세요 직장인부업 블로그 부업 네이버블로그,"[0.02812519483268261, -0.031949013471603394, -...",2,0.313482
27,직장인이 부동산 부자되는 궁극의 7단계 전략,"직장인투자,부동산,부동산강의,투자관리,부자되는법,투자방법,부동산투자,부자되기",INVESTMENT,직장인이 부동산 부자되는 궁극의 7단계 전략 직장인투자 부동산 부동산강의 투자관리 ...,"[0.02064657025039196, -0.043829575181007385, -...",2,0.437975


In [33]:
cluster = articles_df.loc[articles_df['cluster'] == 1]
cluster

,title,tags,category,processed_text,embedding,cluster,similarity
4,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,"전망,웹소설,성공비결,공동창작,콘텐츠,성공공식,집단창작",SIDE_JOB,웹소설 전망 앞으로 웹소설을 써서 성공하기는 힘들 겁니다 전망 웹소설 성공비결 공동...,"[-0.028324633836746216, -0.04125825688242912, ...",1,0.213882
5,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,"세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품",SIDE_JOB,웹소설로 돈 벌려면 3가지를 익히셔야 합니다 세컨드잡 웹소설 성공비결 N잡러 N잡 ...,"[0.024982821196317673, -0.05572228133678436, -...",1,0.302706
6,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림","웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품",SIDE_JOB,웹소설 쓰면 나아지는지 보여드림 웹소설 부업 성공비결 N잡 N잡러 성공공식 인기작품,"[-0.00033927703043445945, -0.04072661325335502...",1,0.326686
7,웹소설을 같이 쓴다는 것,"1등,웹소설,카카오페이지",SIDE_JOB,웹소설을 쓴다는 1등 웹소설 카카오페이지,"[0.0042601414024829865, -0.03769685700535774, ...",1,0.159321
8,카카오페이지 웹소설 1위 찍다!,"1등,웹소설,카카오페이지,인기작품",SIDE_JOB,카카오페이지 웹소설 1위 찍다 1등 웹소설 카카오페이지 인기작품,"[0.005640406161546707, -0.049370430409908295, ...",1,0.184464
11,[1탄] 점만 100번 이상 본 무당 매니아가 말하는 ‘점집 찾는 법’,"신,무당,점린이,점사,신점,사주,점집,미스테리,사주팔자",MYSTERY,1탄 점만 100번 본 무당 매니아가 말하는 점집 찾는 법 신 무당 점린이 점사 신...,"[0.03527821972966194, -0.021226506680250168, -...",1,0.196174
14,"조직 밖에서 스스로 돈벌기, 절반의 성공과 절반의 실패","일러스트레이터,부업,자기계발,N잡,커리어,수익화,디자이너,자기개발,프리랜서",CAREER,조직 밖에서 스스로 돈벌기 절반의 성공과 절반의 실패 일러스트레이터 부업 자기계발 ...,"[-0.0023533031344413757, -0.00556874368339777,...",1,0.339373
19,판타지 웹소설 쓸 때 참조할만한 스킬 & 아이템 설정집,웹소설,SIDE_JOB,판타지 웹소설 쓸 참조할만한 스킬 아이템 설정집 웹소설,"[-0.005888645071536303, -0.013104402460157871,...",1,0.200628
20,조회수 떡상하는 웹소설 표지 만드는 법 (1),웹소설,SIDE_JOB,조회수 떡상하는 웹소설 표지 만드는 법 1 웹소설,"[-0.000671838759444654, -0.038072336465120316,...",1,0.227578
21,조회수 떡상하는 웹소설 표지 만드는 법 (2),웹소설,SIDE_JOB,조회수 떡상하는 웹소설 표지 만드는 법 2 웹소설,"[0.0007775830454193056, -0.03697919473052025, ...",1,0.235030


In [40]:
cluster = articles_df.loc[articles_df['cluster'] == 0]
cluster

,title,tags,category,processed_text,embedding,cluster,similarity
10,마들렌 만드는 법,"마들렌,베이킹레시피간식초코마들렌",HOBBY,마들렌 만드는 법 마들렌 베이킹레시피간식초코마들렌,"[0.004197106231004, -0.024883665144443512, -0....",0,0.177376
12,"[커리어 1탄] 대 이직의 시대, 프로 이직러라면 알아야할 '연차' 꿀팁","연차,커리어,비법거래소,비법,꿀팁",CAREER,커리어 1탄 대 이직의 시대 프로 이직러라면 알아야할 연차 꿀팁 연차 커리어 비법거...,"[-0.014395487494766712, -0.04711250588297844, ...",0,0.234653
15,마플샵 크리에이터 심사 한번에 합격하는 비법 공개 !,"부업,마플샵,비법거래소,굿즈제작,마플샵크리에이터심사승인비법,클라이밍,클라이밍굿즈",SIDE_JOB,마플샵 크리에이터 심사 한번에 합격하는 비법 공개 부업 마플샵 비법거래소 굿즈제작 ...,"[0.02366757206618786, -0.026595812290906906, -...",0,0.225416
